[![Open All Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/search-entities.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python.git"

In [ ]:
import jsonfrom openalex_api import Configuration, ApiClient,AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApiconfiguration = Configuration(host="https://api.openalex.org")authors_api = AuthorsApi(ApiClient(configuration))concepts_api = ConceptsApi(ApiClient(configuration))funders_api = FundersApi(ApiClient(configuration))institutions_api = InstitutionsApi(ApiClient(configuration))publishers_api = PublishersApi(ApiClient(configuration))sources_api = SourcesApi(ApiClient(configuration))works_api = WorksApi(ApiClient(configuration))

# Search entities## The `search` parameterThe `search` query parameter finds results that Match a given text search. Example:* Get works with search term "dna" in the title, abstract, or fulltext:\  [`https://api.openalex.org/works?search=dna`](https://api.openalex.org/works?search=dna)

In [ ]:
response = works_api.get_works(	search="dna")print(json.dumps(response.to_dict(), indent=2))

When you [search `works`](./../../api-entities/works/search-works.ipynb), the API looks for matches in titles, abstracts, and fulltext. When you [search `concepts`](./../../api-entities/concepts/search-concepts.ipynb), we look in each concept's `display_name` and `description` fields. When you [search `sources`](./../../api-entities/sources/search-sources.ipynb), we look at the `display_name`_,_ `alternate_titles`, and `abbreviated_title` fields. When you [search `authors`](./../../api-entities/authors/search-authors.ipynb), we look at the `display_name` and `display_name_alternatives` fields. When you [search `institutions`](./../../api-entities/institutions/search-institutions.ipynb), we look at the `display_name`, `display_name_alternatives`, and `display_name_acronyms` fields.For most text search we remove [stop words](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-stop-tokenfilter.html) and use [stemming](https://en.wikipedia.org/wiki/Stemming) (specifically, the [Kstem token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-kstem-tokenfilter.html)) to improve results. So words like "the" and "an" are transparently removed, and a search for "possums" will also return records using the word "possum." With the exception of raw affiliation strings, we do not search within words but rather try to Match whole words. So a search with "lun" will not Match the word "lunar".### Boolean searches

In [ ]:
response = works_api.get_works(	search="")print(json.dumps(response.to_dict(), indent=2))

In [ ]:
response = works_api.get_works(	search="(elmo AND "sesame street") NOT (cookie OR monster)")print(json.dumps(response.to_dict(), indent=2))

Including any of the words `AND`, `OR`, or `NOT` in any of your searches will enable boolean search. Those words must be UPPERCASE. You can use this in all searches, including using the `search` parameter, and using [search filters](./README.md#the-search-filter).This allows you to craft complex queries using those boolean operators along with parentheses and quotation marks. Surrounding a phrase with quotation marks will search for an exact Match of that phrase, after stemming and stop-word removal (be sure to use **double quotation marks** — `"`). Using parentheses will specify order of operations for the boolean operators. Words that are not separated by one of the boolean operators will be interpreted as `AND`.Behind the scenes, the boolean search is using Elasticsearch's [query string query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html) on the searchable fields (such as title, abstract, and fulltext for works; see each individual entity page for specifics about that entity). Wildcard and fuzzy searches using `*`, `?` or `~` are not allowed; these characters will be removed from any searches. These searches, even when using quotation marks, will go through the same cleaning as desscribed above, including stemming and removal of stop words.* Search for works that mention "elmo" and "sesame street," but not the words "cookie" or "monster": [`https://api.openalex.org/works?search=(elmo AND "sesame street") NOT (cookie OR monster)`](https://api.openalex.org/works?search=%28elmo%20AND%20%22sesame%20street%22%29%20NOT%20%28cookie%20OR%20monster%29)## Relevance scoreWhen you use search, each returned entity in the results lists gets an extra property called `relevance_score`, and the list is by default sorted in descending order of `relevance_score`. The `relevance_score` is based on text similarity to your search term. It also includes a weighting term for citation counts: more highly-cited entities score higher, all else being equal.If you search for a multiple-word phrase, the algorithm will treat each word separately, and rank results higher when the words appear close together. If you want to return only results where the exact phrase is used, just enclose your phrase within quotes. Example:* Get works with the exact phrase "fierce creatures" in the title or abstract (returns just a few results):\  [`https://api.openalex.org/works?search="fierce%20creatures"`](https://api.openalex.org/works?search=%22fierce%20creatures%22)

In [ ]:
response = works_api.get_works(	search="")print(json.dumps(response.to_dict(), indent=2))

In [ ]:
response = works_api.get_works(	search=""fierce creatures"")print(json.dumps(response.to_dict(), indent=2))

* Get works with the words "fierce" and "creatures" in the title or abstract, with works that have the two words close together ranked higher by `relevance_score` (returns way more results):\  [`https://api.openalex.org/works?search=fierce%20creatures`](https://api.openalex.org/works?search=fierce%20creatures)

In [ ]:
response = works_api.get_works(	search="fierce creatures")print(json.dumps(response.to_dict(), indent=2))

## The search filterYou can also use search as a [filter](./filter-entity-lists.ipynb), allowing you to fine-tune the fields you're searching over. To do this, you append `.search` to the end of the property you are filtering for:* Get authors who have "Einstein" as part of their name:\  [`https://api.openalex.org/authors?filter=display_name.search:einstein`](https://api.openalex.org/authors?filter=display\_name.search:einstein)

In [ ]:
response = authors_api.get_authors(	filter="display_name.search:einstein")print(json.dumps(response.to_dict(), indent=2))

* Get works with "cubist" in the title:\  [`https://api.openalex.org/works?filter=title.search:cubist`](https://api.openalex.org/works?filter=title.search:cubist)

In [ ]:
response = works_api.get_works(	filter="title.search:cubist")print(json.dumps(response.to_dict(), indent=2))

Additionally, the filter `default.search` is available on all entities; this works the same as the [`search` parameter](./README.md#the-search-parameter).{% hint style="info" %}You might be tempted to use the search filter to power an autocomplete or typeahead. Instead, we recommend you use the [autocomplete endpoint](./autocomplete-entities.ipynb), which is much faster.\\👎 [`https://api.openalex.org/institutions?filter=display_name.search:florida`](https://api.openalex.org/institutions?filter=display\_name.search:florida)

In [ ]:
response = institutions_api.get_institutions(	filter="display_name.search:florida")print(json.dumps(response.to_dict(), indent=2))

👍 [`https://api.openalex.org/autocomplete/institutions?q=Florida`](https://api.openalex.org/autocomplete/institutions?q=Florida){% endhint %}

In [ ]:
response = autocomplete_api.get_autocomplet(	q="Florida",	id="institutions")print(json.dumps(response.to_dict(), indent=2))